Go through all subjects from the dataset, read the EDF files and store them into NumPy arrays.

**Notes**

* In addition to the module's dependacies, please consult the file `requirements.txt` found in the current folder. 

* In some subjects, we drop the last 170 samples, to make sure equal number of samples across subjects.

In [ ]:
import numpy as np
import pyedflib # please check the "requirements.txt" file
import tqdm
import pathlib

# Fetch the dataset 

Define the directory where dataset is located 

In [ ]:
curr_dir = pathlib.Path("./")

In [ ]:
edf_dir = (curr_dir / "raw_data/").resolve()

In [ ]:
if not edf_dir.exists():
    try:
        edf_dir.mkdir()
    except Exeption as err:
        print(err)
else:
    print(f"\"{edf_dir}\" already exists.")

In [ ]:
!wget -P "$edf_dir" -c https://physionet.org/static/published-projects/eegmmidb/eeg-motor-movementimagery-dataset-1.0.0.zip

In [ ]:
!unzip "$edf_dir"/eeg-motor-movementimagery-dataset-1.0.0.zip -d "$edf_dir/eeg-motor-movementimagery-dataset/"

# Prepare dataset

In [ ]:
dataset_root = f"{edf_dir}/eeg-motor-movementimagery-dataset/files"

In [ ]:
n_subjects = 106
n_rois = 64
n_samples = 9600

In [ ]:
eyes_open = np.zeros((n_subjects, n_rois, n_samples))
eyes_closed = np.zeros((n_subjects, n_rois, n_samples))

## Parse the baseline files for "eyes open" 

In [ ]:
for sub_id in tqdm.tqdm(range(n_subjects)):
    subj_prefix = f"S{sub_id + 1:03}"
    subj_dir = f"{dataset_root}/{subj_prefix}"
    
    baseline_eyes_open = f"{subj_dir}/{subj_prefix}R01"
    
    edf = pyedflib.EdfReader(baseline_eyes_open + ".edf")
    annot = edf.read_annotation()
    n_signals = edf.signals_in_file
    signal_labels = edf.getSignalLabels()

    for chan in np.arange(n_signals):
        eyes_open[sub_id, chan, :] = edf.readSignal(chan)[0:9600]

## Parse the baseline files for "eyes closed" 

In [ ]:
for sub_id in tqdm.tqdm(range(n_subjects)):
    subj_prefix = f"S{sub_id + 1:03}"
    subj_dir = f"{dataset_root}/{subj_prefix}"
    
    baseline_eyes_open = f"{subj_dir}/{subj_prefix}R02"
    
    edf = pyedflib.EdfReader(baseline_eyes_open + ".edf")
    annot = edf.read_annotation()
    n_signals = edf.signals_in_file
    signal_labels = edf.getSignalLabels()

    for chan in np.arange(n_signals):
        eyes_closed[sub_id, chan, :] = edf.readSignal(chan)[0:9600]

## Dump arrays 

In [ ]:
store_dir = (curr_dir / "data/").resolve()

In [ ]:
if not store_dir.exists():
    try:
        store_dir.mkdir()
    except Exeption as err:
        print(err)
else:
    print(f"\"{store_dir}\" already exists.")

In [ ]:
np.save(f'{store_dir}/eeg_eyes_opened.npy', eyes_open)
np.save(f'{store_dir}/eeg_eyes_closed.npy', eyes_closed)